# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	8437 candidates
Dev set:	920 candidates
Test set:	4698 candidates


# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

**Note: Again, training for more epochs than below will greatly improve performance- try it out!**

In [3]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

In [4]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

In [5]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':              0.01,
    'embedding_dim':   100,
    'hidden_dim':      100,
    'n_epochs':        100,
    'dropout':         0.5,
    'rebalance':       0.25,
    'print_freq':      5,
    'seed':            1701
}

lstm = LSTM(n_threads=None)
lstm.train(train, train_marginals, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=2705  #epochs=100  batch size=64
[LSTM] Epoch 1 (5.64s)	Average loss=0.673879	Dev F1=49.96
[LSTM] Epoch 6 (35.53s)	Average loss=0.644199	Dev F1=50.52
[LSTM] Epoch 11 (63.03s)	Average loss=0.638185	Dev F1=51.15
[LSTM] Epoch 16 (90.12s)	Average loss=0.637319	Dev F1=55.31
[LSTM] Epoch 21 (117.16s)	Average loss=0.633461	Dev F1=52.65
[LSTM] Epoch 26 (143.96s)	Average loss=0.634923	Dev F1=51.65
[LSTM] Epoch 31 (170.53s)	Average loss=0.633968	Dev F1=52.67
[LSTM] Epoch 36 (197.35s)	Average loss=0.633124	Dev F1=51.94
[LSTM] Epoch 41 (224.02s)	Average loss=0.632171	Dev F1=51.34
[LSTM] Epoch 46 (250.53s)	Average loss=0.632254	Dev F1=51.38
[LSTM] Epoch 51 (277.17s)	Average loss=0.631974	Dev F1=51.92
[LSTM] Epoch 56 (303.73s)	Average loss=0.633372	Dev F1=51.18
[LSTM] Epoch 61 (330.23s)	Average loss=0.633530	Dev F1=52.81
[LSTM] Epoch 66 (356.74s)	Average loss=0.632895	Dev F1=52.33
[LSTM] Epoch 71 (383.43s)	Average loss=0.637030	Dev F1=51.83
[LSTM] Epoch 76 (409.8

### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [6]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

AnnotatorLabels created: 4698


<4698x1 sparse matrix of type '<class 'numpy.int64'>'
	with 4698 stored elements in Compressed Sparse Row format>

In [7]:
lstm.score(test, L_gold_test)

(0.366376969038566, 0.8875, 0.518646674356017)

In [8]:
tp, fp, tn, fn = lstm.error_analysis(session, test, L_gold_test, set_unlabeled_as_neg=False)

Scores (Un-adjusted)
Pos. class accuracy: 0.887
Neg. class accuracy: 0.266
Precision            0.366
Recall               0.887
F1                   0.519
----------------------------------------
TP: 1349 | FP: 2333 | TN: 845 | FN: 171



In [9]:
for metric in [tp, fp, tn, fn]:
    count = 0
    for sample in metric:
        if sample.chemical.sentence.position != sample.disease.sentence.position:
            count += 1
    print(str(count))

0
0
0
0
